In [1]:
import numpy as np
import tensorflow as tf

#Load and Prepare the MNIST Dataset

In [2]:
# Load the MNIST dataset and split it into training and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Convert 0/255 to 0/1
x_train_int = [np.round(1.0 * i / 256) for i in x_train]
x_test_int = [np.round(1.0 * i / 256) for i in x_test]

11490434/11490434 [==============================] - 0s 0us/step


#Bayes Net Classifier

##Calcualte $p(x_i=0|x_j,x_k,y)$ and $p(x_i=1|x_j,x_k,y)$

In [23]:
pixel_counts = np.zeros([2, 10, 4, 28, 28])
totals = np.zeros([10, 4 ,28, 28])

# Pad the input to handle boundary conditions for conditional probabilities 
x_train_padded = np.pad(np.array(x_train_int), [(0, 0), (1, 0), (1, 0)], mode='constant', constant_values=0)

# Loop over all the training samples
for i in range(np.shape(x_train_padded)[0]):
  # Loop over the rows excluding padding
  for j in range(1, np.shape(x_train_padded)[1]):
    # Loop over the columns excluding padding
    for k in range(1, np.shape(x_train_padded)[2]):
      # Get the value of the pixel to the left of the current pixel
      left_pixel = x_train_padded[i, j, k-1]
      # Get the value of the pixel above the current pixel
      above_pixel = x_train_padded[i, j-1, k]
      # Check is left and above pixel both equal 0
      if left_pixel == 0 and above_pixel == 0:
        totals[y_train[i], 0, j-1, k-1] += 1
        # Check if the current pixel equals 1
        if x_train_padded[i, j, k] == 1:
          pixel_counts[1, y_train[i], 0, j-1, k-1] += 1
      # Check if left pixel equals 0 and above pixel equals 1
      elif left_pixel == 0 and above_pixel == 1:
        totals[y_train[i], 1, j-1, k-1] += 1
        #Check if the current pixel equals 1
        if x_train_padded[i, j, k] == 1:
          pixel_counts[1, y_train[i], 1, j-1, k-1] += 1
      # Check if left pixel equals 1 and above pixel equals 0
      elif left_pixel == 1 and above_pixel == 0:
        totals[y_train[i], 2, j-1, k-1] += 1
        # Check if the current pixel equals 1
        if x_train_padded[i, j, k] == 1:
          pixel_counts[1, y_train[i], 2, j-1, k-1] += 1
      #Check if left and above pixel both equal 1
      elif left_pixel == 1 and above_pixel == 1:
        totals[y_train[i], 3, j-1, k-1] += 1
        # Check if the current pixel equals 1
        if x_train_padded[i, j, k] == 1:
          pixel_counts[1, y_train[i], 3, j-1, k-1] += 1

# Calculate the 0 pixel counts from the 1 pixel counts
pixel_counts[0, :, :, :, :] = totals - pixel_counts[1, :, :, :, :]

# Calculate the probabilities and add Laplace smoothing
p_xi_y = (pixel_counts + 1) / (totals + 2)

# Calculate the log probabilities
log_p_xi_y = np.log10(p_xi_y)

##Bayes Net classification function

In [42]:
def bayes_net(_x_test):
  p_x_test = np.zeros([10, 1])
  # Loop over all classes
  for i in range(10):
    # Loop over rows excluding padding
    for j in range(1, np.shape(_x_test)[0]):
      # Loop over columns excluding padding
      for k in range(1, np.shape(_x_test)[1]):
        # Get the value of the pixel to the left of the current pixel
        left_pixel = _x_test[j, k-1]
        # Get the value of the pixel above the current pixel
        above_pixel = _x_test[j-1, k]
        # Check if current pixel equals 1 and left and above pixel both equal 0
        if _x_test[j, k] == 1 and left_pixel == 0 and above_pixel == 0:
          p_x_test[i] += log_p_xi_y[1, i, 0, j-1, k-1]
        # Check if current pixel equals 1 and left pixel equals 0 and above pixel equals 1
        elif _x_test[j, k] == 1 and left_pixel == 0 and above_pixel == 1:
          p_x_test[i] += log_p_xi_y[1, i, 1, j-1, k-1]
        # Check if current pixel equals 1 and left pixel equals 1 and above pixel equals 0
        elif _x_test[j, k] == 1 and left_pixel == 1 and above_pixel == 0:
          p_x_test[i] += log_p_xi_y[1, i, 2, j-1, k-1]
        # Check if current pixel equals 1 and left and above pixel both equal 1
        elif _x_test[j, k] == 1 and left_pixel == 1 and above_pixel == 1:
          p_x_test[i] += log_p_xi_y[1, i, 3, j-1, k-1]
        # Check if current pixel equals 0 and left and above pixel both equal 0
        elif _x_test[j, k] == 0 and left_pixel == 0 and above_pixel == 0:
          p_x_test[i] += log_p_xi_y[0, i, 0, j-1, k-1]
        # Check if current pixel equals 0 and left pixel equals 0 and above pixel equals 1
        elif _x_test[j, k] == 0 and left_pixel == 0 and above_pixel == 1:
          p_x_test[i] += log_p_xi_y[0, i, 1, j-1, k-1]
        # Check if current pixel equals 0 and left pixel equals 1 and above pixel equals 0
        elif _x_test[j, k] == 0 and left_pixel == 1 and above_pixel == 0:
          p_x_test[i] += log_p_xi_y[0, i, 2, j-1, k-1]
        # Check if current pixel equals 0 and left and above pixel both equal 1
        elif _x_test[j, k] == 0 and left_pixel == 1 and above_pixel == 1:
          p_x_test[i] += log_p_xi_y[0, i, 3, j-1, k-1]
  y = np.argmax(p_x_test)
  return y

##Predict the test classes

In [43]:
x_test_padded = np.pad(np.array(x_test_int), [(0, 0), (1, 0), (1, 0)], mode='constant', constant_values=0)
y_hat = []

for i in range(10000):
  y_hat.append(bayes_net(x_test_padded[i]))

print('The empirical risk when using the missclassification loss function =', np.count_nonzero(y_hat - y_test) / 10000)
print('The empirical risk when using the squared error loss function =', sum((y_hat - y_test)**2) / 10000)

The empirical risk when using the missclassification loss function = 0.0728
The empirical risk when using the squared error loss function = 1.2504
